In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [9]:
recordings = os.listdir("Recordings/training")

In [14]:
df = pd.DataFrame()
df["filenames"] = recordings
df["labels"] = [recording.split('.')[0] for recording in recordings]

In [16]:
print(df)
df.to_csv("labels.csv")

                                             filenames   labels
0     Apple.5kh0pmfj.ingestion-8474d8fb45-pdwsh.s5.wav    Apple
1    Cherry.5kh1qkbd.ingestion-8474d8fb45-9wskt.s5.wav   Cherry
2     Apple.5kh0cn3e.ingestion-8474d8fb45-9wskt.s1.wav    Apple
3    Cherry.5kh2800p.ingestion-8474d8fb45-6skpk.s8.wav   Cherry
4    Cherry.5kh21qib.ingestion-8474d8fb45-pdwsh.s4.wav   Cherry
..                                                 ...      ...
237  Orange.5kh3h8ol.ingestion-8474d8fb45-6skpk.s8.wav   Orange
238  Orange.5kh48ndd.ingestion-8474d8fb45-pdwsh.s4.wav   Orange
239  Orange.5kh32m5q.ingestion-8474d8fb45-9wskt.s7.wav   Orange
240   Apple.5kh10n2n.ingestion-8474d8fb45-pdwsh.s5.wav    Apple
241  Unknown.5kh4s5kb.ingestion-8474d8fb45-pdwsh.s4...  Unknown

[242 rows x 2 columns]
